In [2]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import config

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    

In [4]:
features_importance = pd.read_csv(config.IN_FEAT_IMPORTANCE, index_col=0)
columns = features_importance[:160].index.tolist()

In [5]:
train_features_dl = pd.read_csv(config.IN_TRAIN_FEAT_DL, index_col=0)
train_features_hand = pd.read_csv(config.IN_TRAIN_FEAT_HAND, index_col=0, usecols=['id'] + columns)
train_features = pd.concat([train_features_dl, train_features_hand], axis=1)

In [10]:
x = train_features.values
np.save('train_wxh.npy', x)

In [11]:
test_features_dl = pd.read_csv(config.IN_TEST_FEAT_DL, index_col=0)
test_features_hand = pd.read_csv(config.IN_TEST_FEAT_HAND, index_col=0, usecols=['test_id'] + columns)
test_features = pd.concat([test_features_dl, test_features_hand], axis=1)


In [12]:
x = test_features.values
np.save('test_wxh.npy', x)

In [11]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import config
features_importance = pd.read_csv(config.FEAT_IMPORTANCE, index_col=0)
def select_features(i):
    return features_importance[:i].index.tolist()

In [28]:
for i, x in enumerate(features_importance.index.tolist()):
    if x =='nolemmatized_uniwords_norm-wmd':
        print(i)

74


In [7]:
print_full(features_importance['Random Forest'].sort_values(ascending=False))

lemmatized_glove_dl_SelfAttentionBiLSTM                     1.000
lemmatized_word2vec_dl_SelfAttentionBiLSTM                  0.595
nolemmatized_dl_SelfAttentionBiLSTM                         0.568
magic_intersect                                             0.387
lcs_dl_SelfAttentionBiLSTM                                  0.143
magic_max-freq                                              0.079
lemmatized_dl_SelfAttentionBiLSTM                           0.031
lemmatized_glove_stemmed_dl_SelfAttentionBiLSTM             0.023
lemmatized_stemmed_dl_SelfAttentionBiLSTM                   0.021
lemmatized_word2vec_stemmed_dl_SelfAttentionBiLSTM          0.020
lemmatized_glove_stemmed_chars-tfidf-lsa-cosine             0.014
lemmatized_word2vec_chars-tfidf-lsa-cosine                  0.014
lemmatized_word2vec_stemmed_chars-tfidf-lsa-cosine          0.014
lemmatized_word2vec_words-tfidf-lsa-cosine                  0.014
nolemmatized_fuzz-token-sort-ratio                          0.014
lemmatized

In [ ]:
train_features = pd.read_csv(config.FEAT_TRAIN_DATA, index_col=0, usecols=['id']+select_features(150))
train_features = train_features.astype(dtype=float)

train_label = pd.read_csv(config.TRAIN_LABEL, index_col=0)
train_label = train_label.is_duplicate

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
data = train_features.fillna(0)
label = train_label
dev_ratio = 0.02

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [41]:
from sklearn.ensemble import GradientBoostingClassifier

GradientBoostingClassifier

sklearn.ensemble.gradient_boosting.GradientBoostingClassifier

In [45]:
# lgb data
from sklearn.metrics import log_loss
X_train, X_dev, y_train, y_dev = train_test_split(data, label, test_size=dev_ratio)
clf=RandomForestClassifier(n_estimators=400, verbose=1 )
clf.fit(X_train, y_train)
loss = log_loss(y_dev, clf.predict_proba(X_dev))

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed: 28.1min finished
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    2.1s finished


In [46]:
loss

0.19611818560500327

In [18]:
[x  for x in select_features(150) if 'triwords' in x]

['lemmatized_stemmed_triwords_dice-ratio',
 'lemmatized_word2vec_stemmed_triwords_edit-set-ratio',
 'lemmatized_stemmed_triwords_jaccard-ratio',
 'lemmatized_glove_stemmed_triwords_edit-seq-ratio']

In [22]:
features_importance[120:121]

,Correlation,Lasso,LightGBM,Mean,Random Forest,Ridge
lemmatized_word2vec_stemmed_biwords_edit-seq-ratio,0.116,0.0,0.022,0.067209,0.002,0.193


In [20]:
[(features_importance.at[x, 'Mean'], x) for x in features_importance.index if '' in x ]

[(0.21639901207464327, 'lemmatized_stemmed_bichars_jaccard-ratio'),
 (0.1308280278082693, 'lemmatized_bichars_jaccard-ratio'),
 (0.10318514452982072, 'lemmatized_word2vec_stemmed_bichars_dice-ratio'),
 (0.096274423710208556, 'lemmatized_word2vec_stemmed_bichars_edit-seq-ratio'),
 (0.09403860226856936, 'lemmatized_glove_stemmed_bichars_dice-ratio'),
 (0.089566959385290898, 'lemmatized_stemmed_bichars_dice-ratio'),
 (0.087534394438346139, 'lemmatized_word2vec_bichars_jaccard-ratio'),
 (0.072899926820343949, 'lemmatized_glove_stemmed_bichars_jaccard-ratio'),
 (0.06863154043175998, 'lemmatized_glove_stemmed_bichars_edit-set-ratio'),
 (0.063550128064398095, 'lemmatized_glove_bichars_dice-ratio'),
 (0.059078485181119647, 'lemmatized_stemmed_bichars_edit-set-ratio'),
 (0.058265459202341753, 'lemmatized_word2vec_bichars_edit-seq-ratio'),
 (0.058062202707647274, 'lemmatized_word2vec_bichars_dice-ratio'),
 (0.056639407244785951, 'lemmatized_glove_stemmed_bichars_edit-seq-ratio'),
 (0.05643615075

In [8]:
[x for x in features_importance]

['Correlation', 'Lasso', 'LightGBM', 'Mean', 'Random Forest', 'Ridge']